# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
# lat_lon = [39.678456222120396, -104.92219658639499]

# Perry and 32nd
# lat_lon = [39.76207714280117, -105.03931855940714]

# W 23rd Ave and Julian
lat_lon = [39.751187102730654, -105.03155988650708]

In [4]:
# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.05

query = f"""
select
*
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,1,0,0,0,0
2015,1,0,0,0,0
2021,2,0,0,0,0


In [6]:
display_cols = [
    'crash_date'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
    , 'distance_feet'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [7]:
nearby[display_cols] #.tail(5)

,crash_date,top_traffic_accident_offense,incident_address_corrected,bicycle_ind,pedestrian_ind,sbi,fatality,distance_feet
0,2013-12-18,TRAF - ACCIDENT,W 23RD AVE / N JULIAN ST,0.0,0.0,False,False,11.770261
1,2015-06-02,TRAF - ACCIDENT - HIT & RUN,2200 BLOCK N JULIAN ST / W 23RD AVE,0.0,0.0,False,False,217.451056
2,2021-01-20,TRAF - ACCIDENT,N JULIAN ST / W 23RD AVE,0.0,0.0,False,False,11.770261
3,2021-04-11,TRAF - ACCIDENT,3400 BLOCK W 23RD AVE,0.0,0.0,False,False,85.617910


In [8]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [9]:
cda.conn.close()